In [1]:
library(grf)
library(future.apply)
library(xtable)
plan(multiprocess, workers = 12)


Loading required package: future


In [5]:
run_simulation <- function(param.set,dgp,uniform,iterations) {
    
    repeat_simulation <- function(params,dgp,uniform,iterations) {

        mse.tau <- future_replicate(iterations,run_iteration(params,dgp,uniform)) 
        
        write.table(rowMeans(mse.tau), file = "simulations_smooth.txt", append = TRUE, sep = " ",
             row.names = FALSE,col.names = FALSE)
        return(rowMeans(mse.tau)) 
    }
              
    mse.tau.table <- apply(param.set,MARGIN=1,FUN=repeat_simulation,dgp,uniform,iterations)
    return(t(mse.tau.table)) 
}

run_iteration <- function(params,dgp,uniform) {
    n = params[1]
    p = params[2] 
    noise = params[3]
    if (!uniform){
        X <- matrix(rnorm(n * p), n, p)
    } 
    else { 
        X <- matrix(runif(n*p),n,p)
    }
    data <- dgp(X)
    Y <- data$E.Y + rnorm(n,sd=noise)
    W <- data$W 
    TAU <- data$tau 
    tau.forest.boost = causal_forest(X, Y, W,
                           tune.parameters = TRUE,boosting=TRUE)
    tau.forest = causal_forest(X, Y, W,
                          tune.parameters = TRUE,boosting=FALSE)

    tau.hat.boost = predict(tau.forest.boost)$predictions
    mse.tau.boost = mean((TAU - tau.hat.boost)^2)

    tau.hat.forest = predict(tau.forest)$predictions
    mse.tau.forest = mean((TAU - tau.hat.forest)^2)
    return(c(mse.tau.boost,mse.tau.forest))
}

dgp_nonlinear <- function(X) {
    result = list() 
    TAU = 1 / (1 + exp(-X[, 3]))
    E.W = 1 / (1 + exp(-X[, 1] - X[, 2]))
    W = rbinom(length(E.W) ,1, E.W)
    E.Y = pmax(X[, 2] + X[, 3], 0) + rowMeans(X[, 4:6]) / 2 + W * TAU
    result$W <- W
    result$tau <- TAU
    result$E.Y <- E.Y 
    return(result)
}

dgp_smooth <- function(X) { 
    n = dim(X)[1]
    result = list() 
    E.W = rep(0.5,n)
    TAU = 1 / (1 + exp(-X[, 3]))
    W = rbinom(length(E.W) ,1, E.W)
    E.Y = W*TAU + log(1+exp(6*X[,1]))
    result$W <- W
    result$tau <- TAU
    result$E.Y <- E.Y 
    return(result)
}

dgp_linear <- function(X) {
    n = dim(X)[1]
    result = list() 
    TAU = 2
    E.W = rep(0.5,n)
    W = rbinom(length(E.W) ,1, E.W)
    E.Y = W*TAU + X[,1] +2*X[,2]
    result$W <- W
    result$tau <- TAU
    result$E.Y <- E.Y 
    return(result)
    
}

In [2]:
set.seed(1) 
n.opt <- c(4000)
#p.opt <- c(6,12,60)
p.opt <- c(6,60)
#noise.opt <- c(1,5,10)
noise.opt <-c(1,10) 

param.set <- expand.grid(n.opt,p.opt,noise.opt)

print(param.set)
#mse.table <- run_simulation(param.set[1:nrow(param.set),],dgp_smooth,uniform=TRUE,100)

#results <- cbind(param.set[1:nrow(param.set),],mse.table)
#colnames(results) <- c("n","p","sigma","mse.tau.boost","mse.tau.forest")

#print(xtable(results))
#mean MSE 
#histogram of both 

  Var1 Var2 Var3
1 4000    6    1
2 4000   60    1
3 4000    6   10
4 4000   60   10


In [8]:
results
print(xtable(results))

n,p,sigma,mse.tau.boost,mse.tau.forest
100,6,1,0.065146490,0.071396692
1000,6,1,0.008580967,0.008480546
100,12,1,0.059795536,0.064691942
1000,12,1,0.010889504,0.010978407
100,60,1,0.065557371,0.074234698
1000,60,1,0.009330531,0.010387833
100,6,5,1.300858653,1.310177904
1000,6,5,0.164887434,0.164714684
100,12,5,1.621635858,1.584963007
1000,12,5,0.167330378,0.198072903


% latex table generated in R 3.5.1 by xtable 1.8-3 package
% Fri Mar 29 12:48:04 2019
\begin{table}[ht]
\centering
\begin{tabular}{rrrrrr}
  \hline
 & n & p & sigma & mse.tau.boost & mse.tau.forest \\ 
  \hline
1 & 100.00 & 6.00 & 1.00 & 0.07 & 0.07 \\ 
  2 & 1000.00 & 6.00 & 1.00 & 0.01 & 0.01 \\ 
  3 & 100.00 & 12.00 & 1.00 & 0.06 & 0.06 \\ 
  4 & 1000.00 & 12.00 & 1.00 & 0.01 & 0.01 \\ 
  5 & 100.00 & 60.00 & 1.00 & 0.07 & 0.07 \\ 
  6 & 1000.00 & 60.00 & 1.00 & 0.01 & 0.01 \\ 
  7 & 100.00 & 6.00 & 5.00 & 1.30 & 1.31 \\ 
  8 & 1000.00 & 6.00 & 5.00 & 0.16 & 0.16 \\ 
  9 & 100.00 & 12.00 & 5.00 & 1.62 & 1.58 \\ 
  10 & 1000.00 & 12.00 & 5.00 & 0.17 & 0.20 \\ 
  11 & 100.00 & 60.00 & 5.00 & 1.22 & 1.26 \\ 
  12 & 1000.00 & 60.00 & 5.00 & 0.16 & 0.16 \\ 
  13 & 100.00 & 6.00 & 10.00 & 4.69 & 4.54 \\ 
  14 & 1000.00 & 6.00 & 10.00 & 0.62 & 0.60 \\ 
  15 & 100.00 & 12.00 & 10.00 & 5.37 & 5.50 \\ 
  16 & 1000.00 & 12.00 & 10.00 & 0.61 & 0.66 \\ 
  17 & 100.00 & 60.00 & 10.00 & 4.91 & 4.7

In [ ]:
write.csv(results,"final_output.csv")

In [17]:
result <- read.csv("final_output.csv")
print(xtable(result[,2:ncol(result)],include.rownames=FALSE))

% latex table generated in R 3.5.1 by xtable 1.8-3 package
% Fri Mar 29 12:49:55 2019
\begin{table}[ht]
\centering
\begin{tabular}{rrrrrr}
  \hline
 & n & p & sigma & mse.tau.boost & mse.tau.forest \\ 
  \hline
1 & 100 &   6 &   1 & 0.07 & 0.07 \\ 
  2 & 1000 &   6 &   1 & 0.01 & 0.01 \\ 
  3 & 100 &  12 &   1 & 0.06 & 0.06 \\ 
  4 & 1000 &  12 &   1 & 0.01 & 0.01 \\ 
  5 & 100 &  60 &   1 & 0.07 & 0.07 \\ 
  6 & 1000 &  60 &   1 & 0.01 & 0.01 \\ 
  7 & 100 &   6 &   5 & 1.30 & 1.31 \\ 
  8 & 1000 &   6 &   5 & 0.16 & 0.16 \\ 
  9 & 100 &  12 &   5 & 1.62 & 1.58 \\ 
  10 & 1000 &  12 &   5 & 0.17 & 0.20 \\ 
  11 & 100 &  60 &   5 & 1.22 & 1.26 \\ 
  12 & 1000 &  60 &   5 & 0.16 & 0.16 \\ 
  13 & 100 &   6 &  10 & 4.69 & 4.54 \\ 
  14 & 1000 &   6 &  10 & 0.62 & 0.60 \\ 
  15 & 100 &  12 &  10 & 5.37 & 5.50 \\ 
  16 & 1000 &  12 &  10 & 0.61 & 0.66 \\ 
  17 & 100 &  60 &  10 & 4.91 & 4.75 \\ 
  18 & 1000 &  60 &  10 & 0.66 & 0.66 \\ 
   \hline
\end{tabular}
\end{table}
